In [0]:
select * from csv.`/Volumes/gizmobox/landing/operational_data/addresses/`

In [0]:
select * from read_files(
  '/Volumes/gizmobox/landing/operational_data/addresses',
  delimiter =>'\t',
  header => True,
  inferschema => True,
  format => 'csv'
  ) 
  order by customer_id ;

In [0]:
create or replace view gizmobox.bronze.v_addresses 
as
select * from read_files(
  '/Volumes/gizmobox/landing/operational_data/addresses',
  delimiter =>'\t',
  header => True,
  inferschema => True,
  format => 'csv'
  ) 
  order by customer_id ;

## Silver schema

In [0]:
select * from
(select 
  customer_id, 
  address_type, 
  address_line_1, 
  city, 
  state, 
  postcode 
from gizmobox.bronze.v_addresses)
pivot
(
  max(address_line_1) as address_line_1,
  max(city) as city,
  max(state) as state,
  max(postcode) as postcode
  for address_type in ('shipping','billing')
)

In [0]:
create table if not exists gizmobox.silver.addresses
as
select * from
(select 
  customer_id, 
  address_type, 
  address_line_1, 
  city, 
  state, 
  postcode 
from gizmobox.bronze.v_addresses)
pivot
(
  max(address_line_1) as address_line_1,
  max(city) as city,
  max(state) as state,
  max(postcode) as postcode
  for address_type in ('shipping','billing')
)